In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from dotenv import load_dotenv

load_dotenv()

documents = SimpleDirectoryReader("data").load_data()
print(documents)
print(documents[0])

[Document(id_='b347f2a9-1c2c-4ff8-957f-05ecbd6ea8fa', embedding=None, metadata={'file_path': 'd:\\Projects\\nlp-bachelor\\llama3\\data\\info1.txt', 'file_name': 'info1.txt', 'file_type': 'text/plain', 'file_size': 130, 'creation_date': '2024-07-20', 'last_modified_date': '2024-07-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='The monarch butterfly or simply monarch (Danaus plexippus) is a milkweed butterfly (subfamily Danainae) in the family Nymphalidae.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='07e392aa-9a8f-4460-9538-bece212aca67', embedding=None, metadata={'file_path': 'd:\\Projects\\nlp-bachelor\\llama

In [21]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

session.get(url)

NameError: name 'url' is not defined

In [5]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=200, chunk_overlap=10)
nodes = text_splitter.get_nodes_from_documents(documents=documents)

In [6]:
nodes

[TextNode(id_='69f75b72-e40e-4bca-ae31-c3082f287986', embedding=None, metadata={'file_path': 'd:\\Projects\\nlp-bachelor\\llama3\\data\\info1.txt', 'file_name': 'info1.txt', 'file_type': 'text/plain', 'file_size': 130, 'creation_date': '2024-07-20', 'last_modified_date': '2024-07-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b347f2a9-1c2c-4ff8-957f-05ecbd6ea8fa', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'd:\\Projects\\nlp-bachelor\\llama3\\data\\info1.txt', 'file_name': 'info1.txt', 'file_type': 'text/plain', 'file_size': 130, 'creation_date': '2024-07-20', 'last_modified_date': '2024-07-20'}, hash='7338c181f08cdd85ec9c6238c8278ff678e912657f6b96127ebaee3540828b0d')}, text='The mo

In [8]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

In [22]:
from llama_index.embeddings.openai import OpenAIEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="llama3",
    base_url="https://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)
Settings.embed_model=ollama_embedding

In [23]:
Settings.llm = Ollama(model="llama3", request_timeout=3000.0)
index = VectorStoreIndex.from_documents(
    documents
)

SSLError: HTTPSConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1000)')))

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What is a monarch butterfly?")

In [ ]:
from llama_index.vector